# Desafio Pandas/NumPy: Delivery de Refeições — Gabarito

Este notebook resolve o desafio usando os arquivos em `dados/`.

Seções:
- Importações e leitura
- EDA (head, tail, info, describe)
- Criação de coluna Receita_Item
- Tratamento de ausentes (Quantidade média, drop de Preco_Unitario nulo)
- Agregações por Item (Quantidade_total, Receita_Total)
- Datas e receita por mês
- Merge com cardápio e receita por Categoria
- Filtros e queries
- KPIs e percentis (NumPy)


In [1]:
import pandas as pd
import numpy as np

pd.options.display.float_format = '{:,.2f}'.format

pedidos_path = 'dados/pedidos.csv'
cardapio_path = 'dados/cardapio.csv'

# Leitura
df_pedidos = pd.read_csv(pedidos_path)
df_cardapio = pd.read_csv(cardapio_path)

df_pedidos.head()


ModuleNotFoundError: No module named 'pandas'

In [ ]:
# EDA
df_pedidos.tail()


In [ ]:
df_pedidos.info()


In [ ]:
df_pedidos.describe(include='all')


In [ ]:
# Receita_Item
df_pedidos['Receita_Item'] = df_pedidos['Quantidade'] * df_pedidos['Preco_Unitario']
df_pedidos.head()


In [ ]:
# Tratamento de ausentes
media_qtd = df_pedidos['Quantidade'].mean()
df_pedidos['Quantidade'] = df_pedidos['Quantidade'].fillna(media_qtd)

df_limpo = df_pedidos.dropna(subset=['Preco_Unitario']).copy()
df_limpo.info()


In [ ]:
# Agregações por Item
agg_item = df_limpo.groupby('Item').agg(
    Quantidade_total=('Quantidade','sum'),
    Receita_Total=('Receita_Item','sum')
).reset_index()
agg_item.sort_values('Quantidade_total', ascending=False).head(5)


In [ ]:
# Top 5 por receita
agg_item.sort_values('Receita_Total', ascending=False).head(5)


In [ ]:
# Datas e receita por mês
df_limpo['Data'] = pd.to_datetime(df_limpo['Data'])
df_limpo['AnoMes'] = df_limpo['Data'].dt.to_period('M').astype(str)
receita_mes = df_limpo.groupby('AnoMes')['Receita_Item'].sum().reset_index()
receita_mes


In [ ]:
# Merge com cardápio e receita por categoria
df_merge = pd.merge(df_limpo, df_cardapio, on='Item', how='left')
receita_categoria = df_merge.groupby('Categoria')['Receita_Item'].sum().reset_index().sort_values('Receita_Item', ascending=False)
receita_categoria


In [ ]:
# Filtros e queries (ex.: Salgados com Quantidade > 10)
filtrados = df_merge.query("Categoria == 'Salgados' and Quantidade > 10")
filtrados.head()


In [ ]:
# KPIs e NumPy
receita_total = df_limpo['Receita_Item'].sum()
num_pedidos = df_limpo['ID_Pedido'].nunique()
itens_totais = df_limpo['Quantidade'].sum()
ticket_medio = receita_total / num_pedidos if num_pedidos else 0

p25_preco, p50_preco, p75_preco = np.percentile(df_limpo['Preco_Unitario'], [25,50,75])
p25_qtd, p50_qtd, p75_qtd = np.percentile(df_limpo['Quantidade'], [25,50,75])

print(f"Receita Total: R$ {receita_total:.2f}")
print(f"Pedidos únicos: {num_pedidos}")
print(f"Itens totais vendidos: {itens_totais:.0f}")
print(f"Ticket médio: R$ {ticket_medio:.2f}")
print(f"Percentis Preco_Unitario: 25%={p25_preco:.2f}, 50%={p50_preco:.2f}, 75%={p75_preco:.2f}")
print(f"Percentis Quantidade: 25%={p25_qtd:.2f}, 50%={p50_qtd:.2f}, 75%={p75_qtd:.2f}")
